In [2]:
import spotipy.util as util
import spotipy
import pandas as pd
import numpy as np
import json
import time
import sys
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objects as go
from collections import OrderedDict
from math import log
import calmap


In [3]:
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot

#plotly setup
init_notebook_mode(connected=True)

pd.set_option('display.max_colwidth', None)

In [18]:
#Generate token - If generated it uses the existing
username = '9cjfei8bx9vq78vel4odrjd3y'
client_id ='f2018fb393c2424dba2844c31e8e2c18'
client_secret = 'f4bb9567dafc4625877be647542fd919'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played user-top-read playlist-read-private user-library-read user-follow-read'

token = util.prompt_for_user_token(username=username, 
scope=scope, 
client_id=client_id,   
client_secret=client_secret,     
redirect_uri=redirect_uri)

sp = spotipy.Spotify(auth=token)

In [5]:
#Get ID for song
import requests
def get_id(track_name: str,artist_name: str, token: str) -> str:
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    params = [
    ('q', track_name + ' ' + artist_name),
    ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        #print(json)
        first_result = json['tracks']['items'][0]
        print(first_result)
        track_id = first_result['id']
        return track_id
    except:
        return None


In [6]:
# Get features using track-id
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        print(features)
        return features[0]
    except:
        return None

In [9]:
#Get Artist ID for song
import requests
def get_artist_id(track_name: str,artist_name: str, token: str) -> str:
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    params = [
    ('q', track_name + ' ' + artist_name),
    ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        #print(json)
        first_result = json['tracks']['items'][0]
        #print(first_result)
        id0 = first_result['artists']
        id1 = id0[0] 
        artist_id = id1['id']
        return artist_id
    except:
        return None

In [ ]:
#### Getting list of artist #####
'''top = pd.read_pickle('top_features.pkl')

artist_list = pd.DataFrame(top.iloc[:,16])
artist_list['id'] = 'id'
artist_list['track'] = top.iloc[:,0]
#artist_list

artist_list['id'] = 'id'
row,col = artist_list.shape 
for i in range(0,row):
    artist_list.iloc[i,1] = get_artist_id(artist_list.iloc[i,2],artist_list.iloc[i,0],token)
#artist_list

artist_list = artist_list.drop_duplicates(subset='id')
artist_list = artist_list.reset_index(drop ='True')
#artist_list

#artist_list = artist_list.rename(columns={'analysis_url':'artist'})
#artist_list

artist_list.to_pickle(r'C:\Users\hp\Documents\ProjectOne\Created_Features\artist_list.pkl')

'''

In [15]:
#Future Use
artist_list = pd.read_pickle('artist_list.pkl')

,artist,id,track
0,Maroon 5,04gDigrS5kc9YWfZHwBETP,One More Night
1,Black Veil Brides,6O7MpKrY91vlCd4Osi6XKs,In The End
2,Green Day,7oPftvlwr6VrsViSDV7fJY,Basket Case
3,Taylor Swift,06HL4z0CvFAxyc27GXpf02,I Knew You Were Trouble.
4,Chris Stapleton,4YLtscXsxbVgi031ovDDdh,Tennessee Whiskey
5,Arctic Monkeys,7Ln80lUS6He07XvHI8qqHH,R U Mine?
6,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,Save Your Tears
7,Little Mix,3e7awlrlDSwF3iM0WBjGMp,Touch
8,U2,51Blml2LZPmy7TTiAg47vQ,Beautiful Day
9,Don Omar,33ScadVnbm2X8kkUqOkC6Z,Danza Kuduro


In [26]:
fus = (sp.artist_top_tracks('04gDigrS5kc9YWfZHwBETP'))

In [34]:
fus['tracks'][2]['album']

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/04gDigrS5kc9YWfZHwBETP'},
   'href': 'https://api.spotify.com/v1/artists/04gDigrS5kc9YWfZHwBETP',
   'id': '04gDigrS5kc9YWfZHwBETP',
   'name': 'Maroon 5',
   'type': 'artist',
   'uri': 'spotify:artist:04gDigrS5kc9YWfZHwBETP'}],
 'external_urls': {'spotify': 'https://open.spotify.com/album/2Auw0pTT6EcQdvHNimhLQI'},
 'href': 'https://api.spotify.com/v1/albums/2Auw0pTT6EcQdvHNimhLQI',
 'id': '2Auw0pTT6EcQdvHNimhLQI',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b273442b53773d50e1b5369bb16c',
   'width': 640},
  {'height': 300,
   'url': 'https://i.scdn.co/image/ab67616d00001e02442b53773d50e1b5369bb16c',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67616d00004851442b53773d50e1b5369bb16c',
   'width': 64}],
 'name': 'V',
 'release_date': '2014-09-02',
 'release_date_precision': 'day',
 'total_tracks': 12,
 'type': 'album',
 'uri': 'sp